In [146]:
from SourceFiles.teproteus import TEProteusAdmin as TepAdmin
import matplotlib.pyplot as plt
import numpy as np

In [147]:
# connect and initialize
admin = TepAdmin() #required to control PXI module
sid = 8 #PXI slot WDS found
inst = admin.open_instrument(slot_id=sid)
resp = inst.send_scpi_query("*IDN?")
print('connected to: ' + resp) # Print *IDN
inst.send_scpi_cmd('*CLS; *RST')

Load TEProteus.dll from : C:\Windows\system32\TEProteus.dll
connected to: slot#: 8,


0

In [148]:
# initialize DAC
sampleRateDAC = 1E9
inst.send_scpi_cmd(':FREQ:RAST {0}'.format(sampleRateDAC))
inst.send_scpi_cmd(':SOUR:INT X2')

0

In [149]:
inst.send_scpi_cmd(':INST:CHAN {0}'.format(1)) 
inst.send_scpi_cmd(':MODE DUC')
inst.send_scpi_cmd(':IQM ONE')
inst.send_scpi_query(':SOUR:NCO:PHAS1 0')
inst.send_scpi_query(':SOUR:NCO:CFR1 1E6')

''

In [150]:
# create waveform
max_dac=65535 # Max Dac
half_dac=max_dac/2 # DC Lev
data_type = np.uint16 # DAC data type

segTime =6400


 

 

dacWaveOn_I = np.ones(segTime)

dacWaveOn_I = dacWaveOn_I * max_dac

dacWaveOn_I = dacWaveOn_I.astype(data_type)



dacWaveOn_Q = dacWaveOn_I

# reshapes for IQIQIQIQIQ....

dacWaveOn_IQ = np.array([val for pair in zip(dacWaveOn_I, dacWaveOn_Q) for val in pair])

In [151]:
# select CH1

ch = 1 # everything after relates to CH 1 
segnum = 1
inst.send_scpi_cmd(':INST:CHAN {0}'.format(ch)) 
inst.send_scpi_cmd(':TRAC:DEL:ALL')
inst.send_scpi_cmd(':TRAC:DEF {0}, {1}'.format(segnum, len(dacWaveOn_IQ)))
inst.send_scpi_cmd(':TRAC:SEL {0}'.format(segnum))
inst.write_binary_data(':TRAC:DATA', dacWaveOn_IQ) # write, and wait while *OPC completes

inst.send_scpi_cmd(':TASK:ZERO:ALL')
inst.send_scpi_cmd(':TASK:COMP:LENG 1')
inst.send_scpi_cmd(':TASK:COMP:SEL 1')
inst.send_scpi_cmd(':TASK:COMP:SEGM 1')


inst.send_scpi_cmd(':TASK:COMP:ENAB CPU')
inst.send_scpi_cmd(':TASK:COMP:NEXT1 1')
inst.send_scpi_cmd(':TASK:COMP:WRITE')
inst.send_scpi_cmd(':SOUR:FUNC:MODE TASK')
inst.send_scpi_cmd(':OUTP ON')

0

In [152]:

inst.send_scpi_cmd('*TRG')


# inst.send_scpi_cmd(':TRIG:SEL TRG1')
# inst.send_scpi_cmd(':TRIG:STAT ON')
# inst.send_scpi_cmd(':TRIG:LEV 1.0')









0